# Transformer

## Data

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence
from collections import Counter

import math
import numpy as np
import re
from tqdm import tqdm
import requests
import os

In [ ]:
url = "https://www.gutenberg.org/files/100/100-0.txt"
response = requests.get(url)
shakespeare_text = response.text

shakespeare_text = re.sub(r'\s+', ' ', shakespeare_text)
hamlet_text = shakespeare_text[927066:1105544]

In [ ]:
def tokenize(text):
    return text.lower().split()

vocab = Counter(tokenize(hamlet_text)) # Counter: 단어의 빈도수를 셈
vocab['<PAD>'] = 0 # 패딩용 단어
vocab['<UNK>'] = 1 # OOV(Out-Of-Vocabulary) 단어
vocab = {word: i+2 for i, (word, _) in enumerate(vocab.items())} # 각 단어에 고유한 정수 부여(2 이상의 정수부터)

def encode(text, vocab):
    # 텍스트를 띄어쓰기 기준으로 토큰화하고, 각 토큰을 정수로 변환
    return [vocab.get(token, vocab['<UNK>']) for token in tokenize(text)] 

In [ ]:
class TextDataset(Dataset):
    def __init__(self, text, vocab, seq_length):
        self.sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text) # 문장 단위로 분리
        self.vocab = vocab
        self.seq_length = seq_length
        self.encoded_sentences = [encode(sentence, vocab) for sentence in self.sentences] # 단어를 정수로 변환
        self.encoded_sentences = [sent for sent in self.encoded_sentences if len(sent) > 1]  # 빈 문장 제거

    def __len__(self):
        # 데이터셋의 총 샘플 수: 전체 문장의 개수(encoded_sentences)
        return 

    def __getitem__(self, idx):
        # idx번째 샘플을 반환
        sentence = self.encoded_sentences[idx]
        # 입력 시퀀스: 문장의 처음부터 seq_length-1까지
        input_seq = 
        # 타깃 시퀀스: 문장의 두 번째 단어부터 seq_length까지
        target_seq = 

        # 문장 길이(len(input_seq))가 seq_length보다 작은 경우 패딩 추가
        if 
            # 부족한 만큼 패딩 추가: (seq_length - len(input_seq), len(target_seq))
            input_seq.extend()
            target_seq.extend()

        # input_seq, target_seq를 torch tensor로 변환
        return 

In [ ]:
def collate_fn(batch):
    '''
    DataLoader에서 미니배치를 처리하는 함수: 패딩을 적용하여 길이를 맞춰줌
    '''
    
    inputs, targets = zip(*batch) # 미니배치에서 각 샘플의 입력과 타깃을 분리
    inputs = pad_sequence(inputs, batch_first=True, padding_value=vocab['<PAD>']) # 입력 시퀀스에 대해 패딩 추가
    targets = pad_sequence(targets, batch_first=True, padding_value=vocab['<PAD>']) # 타깃 시퀀스에 대해 패딩 추가
    return inputs, targets

## Model

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads
        
        # Q, K, V에 해당하는 가중치 행렬 생성
        # nn.Linear로 d_model만큼의 차원의 W_q, W_k, W_v를 만듦
        self.query = 
        self.key = 
        self.value = 
        self.fc = nn.Linear(d_model, d_model)
        
    def forward(self, q, k, v, mask=None):
        # 초기 Q, K, V 입력 차원: (batch_size, seq_len, d_model)
        
        batch_size = q.size(0)
        
        # 1. Linear transformation: Q, K, V에 각각 가중치 행렬을 곱함
        # 2. View & Transpose: multi-head attention을 수행하기 위해 텐서 형태 변환
            # view: (batch_size, seq_len, d_model) -> (batch_size, seq_len, num_heads, d_k)
            # transpose: (batch_size, num_heads, seq_len, d_k) -> (batch_size, num_heads, d_k, seq_len)
            # d_k: 각 head가 담당하는 d_model의 차원 크기
        q = 
        k = 
        v = 
        
        # 3. Scaled Dot Product Attention: 각 head 별로 어텐션 수행
        # (batch_size, num_heads, seq_len, d_k) @ (batch_size, num_heads, d_k, seq_len) -> (batch_size, num_heads, seq_len, seq_len)
        scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(self.d_k)
        
        # 4. Masking
        if mask is not None:
            scores = scores.masked_fill(mask == 0, -1e9)
        
        # 5. Softmax: 각 head 별로 어텐션 스코어(scaled attention score) 계산
        attn = F.
        
        # 6. Weighted Sum: 각 head 별로 V에 대한 어텐션 값(attention value) 계산
        # (batch_size, num_heads, seq_len, seq_len) @ (batch_size, num_heads, seq_len, d_k) -> (batch_size, num_heads, seq_len, d_k)
        context = torch.
        
        # 7. Concatenation: 각 head 별 결과를 하나로 합침
            # transpose: (batch_size, num_heads, seq_len, d_k) -> (batch_size, seq_len, num_heads, d_k)
            # contiguous: 메모리 상에 연속적인 공간에 재배열
            # view: (batch_size, seq_len, num_heads, d_k) -> (batch_size, seq_len, num_heads * d_k = d_model)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, self.d_model)
        
        # 8. Final Linear: multi-head attention의 결과에 대해 최종 가중치 행렬을 곱함
        # (batch_size, seq_len, d_model)
        output = self.fc(context)
        
        return output # (batch_size, seq_len, d_model)

In [ ]:
class PositionwiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        # nn.Linear로 d_model 차원을 d_ff 차원으로 변경
        self.fc1 = 
        # nn.Linear로 d_ff 차원을 d_model 차원으로 변경
        self.fc2 = 
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        # fc1 -> ReLU -> dropout -> fc2
        return 

In [ ]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super(PositionalEncoding, self).__init__()
        pe = torch.zeros(max_len, d_model) # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1) # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model)) # (d_model/2)
        pe[:, 0::2] = torch.sin(position * div_term) # 짝수 인덱스에는 sin 함수 적용
        pe[:, 1::2] = torch.cos(position * div_term) # 홀수 인덱스에는 cos 함수 적용
        pe = pe.unsqueeze(0).transpose(0, 1) # (max_len, 1, d_model) -> (max_len, d_model, 1)
        self.register_buffer('pe', pe) 
        
    def forward(self, x):
        x = x + self.pe[:x.size(0), :] # 입력(x)과 positional encoding(self.pe)를 더함
        
        return x

In [ ]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(EncoderLayer, self).__init__()
        # Multi-Head Attention
        self.self_attn = 
        # Positionwise FeedForward
        self.feed_forward = 
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, mask=None):
        # Multi-Head Attention: self-attention
        # Q, K, V에 입력 x를 넣고, 마스킹은 mask를 사용
        attn_output = 
        # Layer Normalization & Residual Connection
        # 입력 x와 attention 결과를 더하고 Layer Normalization
        x = 
        
        # Positionwise FeedForward
        ff_output = 
        # Layer Normalization & Residual Connection
        # 이전 단계의 출력과 Positionwise FeedForward 결과를 더하고 Layer Normalization
        x = 
        
        return x

In [ ]:
class DecodrLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout=0.1):
        super(DecoderLayer, self).__init__()
        # Multi-Head Attention (self-attention)
        self.self_attn = 
        # Multi-Head Attention (encoder-decoder attention)
        self.cross_attn =
        # Positionwise FeedForward
        self.feed_forward = 
        self.layernorm1 = nn.LayerNorm(d_model)
        self.layernorm2 = nn.LayerNorm(d_model)
        self.layernorm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x, enc_output, src_mask=None, tgt_mask=None):
        # Multi-Head Attention: self-attention
        # Q, K, V에 입력 x를 넣고, 마스킹은 tgt_mask를 사용
        self_attn_output = 
        # Layer Normalization & Residual Connection
        # 입력 x와 self-attention 결과를 더하고 Layer Normalization
        x = 
        
        # Multi-Head Attention: encoder-decoder attention
        # Q: decoder layer의 출력 x, K, V: encoder의 출력 enc_output
        cross_attn_output = 
        # Layer Normalization & Residual Connection
        # 이전 단계의 출력과 encoder-decoder attention 결과를 더하고 Layer Normalization
        x = 
        
        # Positionwise FeedForward
        ff_output = 
        # Layer Normalization & Residual Connection
        # 이전 단계의 출력과 Positionwise FeedForward 결과를 더하고 Layer Normalization
        x = 
        
        return x

In [ ]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model=512, num_heads=8, d_ff=2048, num_layers=6, dropout=0.1):
        super(Transformer, self).__init__()
        # Embedding
        # encoder, decoder의 입력 차원: src_vocab_size, tgt_vocab_size -> d_model
        self.encoder_embedding = 
        self.decoder_embedding = 
        
        # Positional Encoding
        self.positional_encoding = PositionalEncoding(d_model)
        
        # Encoder & Decoder
        # num_layers만큼 EncoderLayer, DecoderLayer를 쌓음
        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = 
        
        # Fully Connected Layer
        self.fc_out = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src, tgt, src_mask=None, tgt_mask=None):
        # Encoder: Embedding & Positional Encoding
        src = 
        
        # Encoder: 입력 문장을 입력받아 인코더 레이어(self.encoder_layers)를 차례대로 거침
        for layer in self.encoder_layers:
            src = layer(src, src_mask)
        
        # Decoder: Embedding & Positional Encoding
        tgt = 
        
        # Decoder: 출력 문장을 입력받아 디코더 레이어(self.decoder_layers)를 차례대로 거침
        for 
        
        # Fully Connected Layer: decoder 출력을 이용해 타겟 단어의 logit을 계산
        output = 
        
        return output

In [ ]:
def average_sentence_length(text):
    '''
    문장의 평균 단어 개수 계산하는 함수
    '''
    sentences = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    total_words = sum(len(tokenize(sentence)) for sentence in sentences)
    average_length = total_words / len(sentences)
    return average_length

average_length = average_sentence_length(hamlet_text)
print(average_length)

In [ ]:
# Hyperparameters
seq_length = 10 # 문장의 길이(단어 개수)
batch_size = 512
embedding_dim = 128
hidden_size = 256
learning_rate = 0.001
epochs = 10

if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')

# Model, Loss, Optimizer 생성
model = 
criterion = 
optimizer = 

# TextDataset 생성
dataset = 

train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size

train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# train_loader, test_loader 생성
train_loader = 
test_loader = 

In [ ]:
def train_and_test(model, train_loader, test_loader, criterion, optimizer, epochs, device):
    # 모델을 device에 할당
    
    
    # epochs만큼 반복
    for 
        # Training
        # model을 train 모드로 설정
        
        # loss 초기화
        
        # train_loader를 이용해 학습
        for inputs, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Training]", leave=False):
            # inputs, targets를 device에 할당
            
            # gradient 초기화
            optimizer.zero_grad()
            # model에 inputs를 넣어 output을 계산
            outputs = 
            
            # loss 계산 및 역전파
            loss = criterion(outputs.view(-1, outputs.size(2)), targets.view(-1))
            loss.
            optimizer.

            # loss 누적
            train_loss
            
        # 평균 train loss 계산(epoch 당)
        avg_train_loss = train_loss / len(train_loader)
        
        # Testing
        # model을 eval 모드로 설정
        
        # loss 초기화
        
        # gradient 계산 비활성화
        
            # test_loader를 이용해 테스트
            for inputs, targets in tqdm(test_loader, desc=f"Epoch {epoch+1}/{epochs} [Testing]", leave=False):
                # inputs, targets를 device에 할당

                # model에 inputs를 넣어 output을 계산
                outputs = model(inputs, inputs)
                # loss 계산 및 누적
                loss = criterion(outputs.view(-1, outputs.size(2)), targets.view(-1))
                test_loss
                
        # 평균 test loss 계산(epoch 당)
        avg_test_loss = 

        print(f'Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Test Loss: {avg_test_loss:.4f}')

# Train the model
train_and_test(model, train_loader, test_loader, criterion, optimizer, epochs, device)

In [ ]:
# Prediction function
def predict(model, start_sequence, vocab, seq_length, max_length=100):
    model.eval()
    tokens = tokenize(start_sequence)
    generated = tokens
    for _ in range(max_length):
        input_seq = torch.tensor([vocab.get(token, vocab['<UNK>']) for token in generated[-seq_length:]], dtype=torch.long).unsqueeze(0).to(device)
        with torch.no_grad():
            output = model(input_seq, input_seq)
        
        next_token = output[:, -1, :].argmax(dim=-1).item()
        
        next_word = [word for word, idx in vocab.items() if idx == next_token][0]
        generated.append(next_word)
        
        if next_word == '<PAD>':
            break

    return ' '.join(generated)

input_text = "to be or not to be that is the question"
predicted_output = predict(model, input_text, vocab, seq_length)
print(predicted_output)